In [1]:
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder

# load artifacts (if you have them)
model = joblib.load("decision_tree_model.joblib")
scaler = None
encoders = None
try:
    scaler = joblib.load("scaler.joblib")
    print("Loaded scaler")
except:
    print("No scaler.joblib found")
try:
    encoders = joblib.load("encoders.joblib")
    print("Loaded encoders:", list(encoders.keys()))
except:
    print("No encoders.joblib found")

# the example (use the values you pasted)
example = {
    'no_of_dependents': 2,
    'education': 'Graduate',
    'self_employed': 'No',
    'income_annum': 9600000,
    'loan_amount': 29900000,
    'loan_term': 12,
    'cibil_score': 778,
    'residential_assets_value': 2400000,
    'commercial_assets_value': 17600000,
    'luxury_assets_value': 22700000,
    'bank_asset_value': 8000000
}

feature_order = [
    'no_of_dependents','education','self_employed','income_annum','loan_amount',
    'loan_term','cibil_score','residential_assets_value','commercial_assets_value',
    'luxury_assets_value','bank_asset_value'
]

# encoding fallback (same as app)
def encode_categorical(value, col_name):
    if encoders and col_name in encoders:
        le = encoders[col_name]
        try:
            return int(le.transform([value])[0])
        except Exception:
            mapping = {label: idx for idx, label in enumerate(le.classes_)}
            if value in mapping:
                return int(mapping[value])
    if col_name == "education":
        return 1 if str(value).lower().startswith("grad") else 0
    if col_name == "self_employed":
        return 1 if str(value).lower() in ("yes","y","true","1") else 0
    return value

row = {
    'no_of_dependents': example['no_of_dependents'],
    'education': encode_categorical(example['education'], 'education'),
    'self_employed': encode_categorical(example['self_employed'], 'self_employed'),
    'income_annum': example['income_annum'],
    'loan_amount': example['loan_amount'],
    'loan_term': example['loan_term'],
    'cibil_score': example['cibil_score'],
    'residential_assets_value': example['residential_assets_value'],
    'commercial_assets_value': example['commercial_assets_value'],
    'luxury_assets_value': example['luxury_assets_value'],
    'bank_asset_value': example['bank_asset_value']
}
row_df = pd.DataFrame([row], columns=feature_order)
print("Row BEFORE scaling:\n", row_df.T)

# if scaler exists, apply only on numeric cols (the same nine you used earlier)
numerical_cols = [
    'no_of_dependents', 'income_annum', 'loan_amount', 'loan_term',
    'cibil_score', 'residential_assets_value', 'commercial_assets_value',
    'luxury_assets_value', 'bank_asset_value'
]
if scaler is not None:
    row_df[numerical_cols] = scaler.transform(row_df[numerical_cols])
    print("Row AFTER scaling (numeric cols):\n", row_df.T)

# show feature dtypes
print("Dtypes:\n", row_df.dtypes)

# show model.feature_names_in_ if available
if hasattr(model, "feature_names_in_"):
    print("Model was trained on these feature names:", model.feature_names_in_)

# predictions & probabilities
pred = model.predict(row_df)
print("model.predict ->", pred)
if hasattr(model, "predict_proba"):
    print("model.predict_proba ->", model.predict_proba(row_df))


Loaded scaler
Loaded encoders: ['education', 'self_employed', 'loan_status']
Row BEFORE scaling:
                                  0
no_of_dependents                 2
education                        1
self_employed                    0
income_annum               9600000
loan_amount               29900000
loan_term                       12
cibil_score                    778
residential_assets_value   2400000
commercial_assets_value   17600000
luxury_assets_value       22700000
bank_asset_value           8000000
Row AFTER scaling (numeric cols):
                                  0
no_of_dependents         -0.294102
education                 1.000000
self_employed             0.000000
income_annum              1.617979
loan_amount               1.633052
loan_term                 0.192617
cibil_score               1.032792
residential_assets_value -0.780058
commercial_assets_value   2.877289
luxury_assets_value       0.832028
bank_asset_value          0.930304
Dtypes:
 no_of_dependents  

In [2]:
import joblib
model = joblib.load("decision_tree_model.joblib")
encoders = joblib.load("encoders.joblib")  # you said this exists

print("model.classes_ ->", model.classes_)
if 'loan_status' in encoders:
    le_target = encoders['loan_status']
    print("encoders['loan_status'].classes_ ->", le_target.classes_)
    # Example: if model predicted 0:
    print("inverse transform of 0 ->", le_target.inverse_transform([0])[0])
    print("inverse transform of 1 ->", le_target.inverse_transform([1])[0])
else:
    print("No target encoder found in encoders.joblib")


model.classes_ -> [0 1]
encoders['loan_status'].classes_ -> ['Approved' 'Rejected']
inverse transform of 0 -> Approved
inverse transform of 1 -> Rejected
